In [2]:
print("all_ok")

all_ok


In [5]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [7]:
model= ChatGroq(model = "qwen/qwen3-32b")

In [8]:
model.invoke("where is new delhi?")

AIMessage(content='<think>\nOkay, the user is asking "where is New Delhi?" I need to provide a clear and concise answer. Let me start by recalling that New Delhi is the capital of India. I should mention its location within India, maybe specify the state it\'s in, which is Delhi. Wait, but Delhi is actually a union territory, right? So I should clarify that. \n\nNext, for someone who might not know much about India\'s geography, it might help to mention neighboring countries or major cities. For example, it\'s near the border with Pakistan, but not too close. Maybe mention states around it, like Haryana and Uttar Pradesh. Also, its position in northern India.\n\nI should include some geographical coordinates to be precise. Let me check the approximate latitude and longitude. I think it\'s around 28.6139° N, 77.2090° E. That\'s correct. Also, the altitude is about 217 meters above sea level.\n\nWhat else? New Delhi is part of the National Capital Territory of Delhi. Maybe explain that i

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
emdedding_model = GoogleGenerativeAIEmbeddings(model ="models/embedding-001")

In [11]:
emdedding_model.embed_query("where is new delhi?")

[0.04433692991733551,
 0.00699573801830411,
 -0.06549712270498276,
 -0.01715826615691185,
 0.048965390771627426,
 0.02333848364651203,
 0.031678300350904465,
 -0.006987929809838533,
 0.010462739504873753,
 0.062008630484342575,
 -0.03259121999144554,
 0.020617589354515076,
 -0.016330992802977562,
 0.019450459629297256,
 0.0037003301549702883,
 -0.01399842556566,
 0.01470889337360859,
 0.006311079952865839,
 0.029109245166182518,
 0.00046806002501398325,
 0.024623390287160873,
 -0.012324889190495014,
 -0.01583213172852993,
 0.01895468682050705,
 0.025217445567250252,
 -0.01996988244354725,
 -0.005269229877740145,
 -0.055423278361558914,
 -0.018035918474197388,
 -0.013850034214556217,
 -0.04238704964518547,
 0.04118114709854126,
 -0.03464901074767113,
 0.0316159650683403,
 0.023362930864095688,
 -0.05730219557881355,
 -0.02638065069913864,
 0.04128924012184143,
 -0.036266107112169266,
 0.03631479665637016,
 0.003299003466963768,
 -0.021644271910190582,
 -0.08695508539676666,
 0.017317313

In [12]:
#1) Data Injection

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [17]:
file_path = os.path.join(os.getcwd(),"data","sample.pdf")
loader = PyPDFLoader(file_path)
documents = loader.load()

In [19]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100,length_function=len)

data_splits = text_spliter.split_documents(documents)

#FAISS is in memory storage

In [21]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(data_splits, emdedding_model)

In [39]:
relevant_docs = vector_store.similarity_search("llam2 finetunning bencmark experiments?")
retriver = vector_store.as_retriever(search_kwargs={"k":10})

In [28]:
relevant_docs[0].page_content

'Learning Representations, 2022.\n44'

In [30]:
relevant_docs[2].page_content

'final learning rate down to 10% of the peak learning rate. We use a weight decay of0.1 and gradient clipping\nof 1.0. Figure 5 (a) shows the training loss forLlama 2with these hyperparameters.\n5'

In [32]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [35]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
prompt = PromptTemplate(template=prompt_template,input_variables=["context", "question"])

In [36]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [41]:
chain = (
        {"context": retriver | format_docs, "question": RunnablePassthrough()}
         |prompt
         |model
         |StrOutputParser())

In [42]:
chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

'<think>\nOkay, let\'s try to figure out what the user is asking here. They want to know about the Llama2 fine-tuning benchmark experiments based on the provided context. \n\nFirst, I\'ll look through the context to see where finetuning is mentioned. The context includes various numbers and tables comparing different models, including Llama 2 at different sizes (7B, 13B, 34B, 70B) and other models like ChatGPT, MPT-instruct, Falcon-instruct. There are several sections with benchmark results, like MMLU, HellaSwag, Winogrande, etc. \n\nLooking at the first part, there\'s a mention of "Fine-tuned" under Llama 2 with different metrics. The numbers seem to represent performance metrics across different benchmarks. For example, under "Fine-tuned" there are entries like ChatGPT with very low numbers compared to Llama 2\'s 7B, 13B, etc. Then there\'s a section about training hyperparameters: final learning rate down to 10% of peak, weight decay of 0.1, gradient clipping of 1.0.\n\nFurther down